In [2]:
import numpy as np
from scipy import io
import sys
import os
# import matplotlib.pyplot as plt
# %matplotlib inline

In [6]:
mat_dict = io.loadmat('data/portfolio.mat')
test_cl = mat_dict['train_cl']
test_hi = mat_dict['train_hi']
test_lo = mat_dict['train_lo']
test_op = mat_dict['train_op']
test_vol = mat_dict['train_vol']
numDays, numStocks = test_cl.shape
test_stocks = []
for i in range(numStocks):
    test_stocks.append(np.array_str(mat_dict['train_stocks'][0][i])[3:-2])

In [155]:
def uniform_rebalance(t, portfolio_t, hi, lo, cl, op, stocks, vol):
    """% Input: 
    %%% t: we are at Day t
    %%% portfolio_t: current portfolio before open of Day t
    %%% hi: highest prices at Day t-1
    %%% lo: lowest prices at Day t-1
    %%% cl: close prices at Day t-1
    %%% op: open prices at Day t
    %%% stocks: names of all stocks
    %%% vol: volumes at Day t-1

    % Output: 
    %%% portfolio: a column vector, each entry represents the portion of 
    %%%            the value of the corresponding stock over the total cash 
    %%%            realizable value [as a result, the value of inactive stocks 
    %%%            i.e., with open price NaN, cannot be added into the total
    %%%            cash realizable value]. Hence, your output must satisfy
    %%%            sum(portfolio(isfinite(op))) < 1, and the rest is the 
    %%%            portion of cash realizable value you hold in cash. 
    %%% hold_flag: a binary vector representing whether you would like to hold
    %%%            (with value 1) each stock or you choose to rebalance it
    %%%            (with value 0). """
    numStocks = len(stocks)
    portfolio = np.zeros(numStocks)
    numActiveStock = np.sum(np.isfinite(op))
    portfolio[np.isfinite(op)] = 1.0/numActiveStock
    if np.mod(t,2) == 1:
        hold_flag = np.zeros(numStocks,dtype=bool)
    else:
        hold_flag = np.ones(numStocks,dtype=bool)      
    return portfolio, hold_flag

In [7]:
def rebalance(value_vec, value_realizable, portfolio_dst, cost_rate):
    """ % This function simulates the process of rebalancing. The key difficulty 
    % here is to compute the transaction cost, i.e., to solve the equation
    %
    % sum( cost_rate * abs( portfolio_dst .* (value_realizable - C) -
    % value_vec) ) = C
    % 
    % where C is the total transcation cost. We use the Banach contraction 
    % theorem to solve this equation, with convergence rate cost_rate^t for 
    % t-th iteration. 

    % Input: 
    %%% value_vec: current values of each active stock
    %%% value_realizable: total cash realizable value
    %%% portfolio_dst: the desired portfolio after rebalancing
    %%% cost_rate: ratio of transactions cost (0.0005 in our case)

    % Output:
    %%% new_value_vec: the value vector after rebalancing
    %%% trans_cost: the total transaction cost"""
    
    if len(portfolio_dst)==0:
        new_value_vec = value_vec
        trans_cost = 0
    else:
        len_noHold = value_vec.size
        C_candidate = np.append([0], value_realizable - value_vec / portfolio_dst)
        x = np.sort(C_candidate)
        x[np.logical_or(np.isinf(x),np.isnan(x))] = 1e15
        x1 = value_realizable - x
        len_x = len(x1)
        
        y = np.sum(cost_rate * np.abs(np.dot(portfolio_dst.reshape(len_noHold,1),x1.reshape(1,len_x)) \
                  -np.dot(value_vec.reshape(len_noHold,1),np.ones((1,len_x)))), axis=0).squeeze() - x
        I = np.where(y<=0)[0]
        if (len(I)==0) or I[0]==0:
            trans_cost = 0
            new_value_vec = value_vec
        else: #  the equation is linear in [C_sort(I-1), C_sort(I)]           
            I = I[0]
            trans_cost = x[I-1] - y[I-1] * (x[I] - x[I-1]) / (y[I] - y[I-1])
            if trans_cost > value_realizable:
                trans_cost = 0
                new_value_vec = value_vec
            else:
                new_value_vec = portfolio_dst * (value_realizable - trans_cost)
    return new_value_vec, trans_cost

In [8]:
value_op_seq = np.zeros(numDays) # total value before open of each day
value_cl_seq = np.zeros(numDays) # total value before close of each day 
portfolio_seq = np.zeros((numStocks, numDays)) # portfolio before open of each day
last_close_price = np.NaN * np.ones(numStocks)
print numDays, numStocks

1247 497


In [9]:
# Initialization
op = test_op[0,:]
cl = np.NaN * np.ones(numStocks)
hi = np.NaN * np.ones(numStocks)
lo = np.NaN * np.ones(numStocks)
vol = np.NaN * np.ones(numStocks)
value_op_seq[0] = 1 # initially we have one unit of cash

In [21]:
from ucrp import UniformConstantRebalancedPortfolio
from ubah import UniformBuyAndHoldPortfolio
from util import load_matlab_sp500_data
from expert_pool import ExpertPool
from olmar import OLMAR
from rmr import RMR
#from nonparametric_markowitz import NonParametricMarkowitz
import numpy as np

# ******TODO:Change this directory to point to the test file*****
directory = 'data/portfolio.mat'

train_start = 760
train_data = load_matlab_sp500_data(directory,start=train_start)
#ucrp = UniformConstantRebalancedPortfolio(market_data=train_data, tune_interval=None, verbose=True, 
#                                          past_results_dir='train_results/UCRP/')
olmar = OLMAR(market_data=train_data, tune_interval=5, verbose=True) #, past_results_dir='train_results/OLMAR/')
rmr = RMR(market_data=train_data, tune_interval=200, verbose=True) #, past_results_dir='train_results/RMR/')
#npm = NonParametricMarkowitz(market_data=train_data, tune_interval=None, verbose=True, past_results_dir='train_results/NPM/')
pool = ExpertPool(market_data=train_data, experts=[olmar, rmr], verbose=True) #, past_results_dir='train_results/EP/')
pool.run()

portfolios = pool.b_history
portfolios = np.concatenate((portfolios, np.reshape(pool.b, (-1,1))), axis=1)

for t in range(numDays):
    # At the beginning of day t, we use all information up to the open 
    # price of day t to determine the capital allocation at the end of 
    # day t.
    
    # Your function should be placed here. Note that the input vector may
    # contain NaNs (at Day 1, or for some inactive stocks). 
    #new_portfolio, hold_flag = uniform_rebalance(t, portfolio_seq[:,t], hi, lo, cl, op, test_stocks, vol)
    new_portfolio = portfolios[:,t+1]
    hold_flag = np.zeros
    
    # Check whether your output is valid
    isActive = np.isfinite(op)
    noHold = np.logical_and(isActive, np.logical_not(hold_flag))
    if np.sum(new_portfolio[noHold]) > 1 + 1e-8:
        sys.stderr.write('The output portfolio is not valid!')
        
    # The value of our portfolio at the end of Day t before paying transaction costs
    cl = test_cl[t,:]
    value_vec = value_op_seq[t] * portfolio_seq[:,t]
    growth = cl[isActive] / last_close_price[isActive]-1
    growth[np.isnan(growth)] = 0
    revenue_vec = value_vec[isActive] * growth
    value_vec[isActive] = value_vec[isActive] + revenue_vec
    value_cl_seq[t] = value_op_seq[t] + np.sum(revenue_vec)
        

    # At the end of Day t, we use the close price of day t to adjust our 
    # portfolio to the desired percentage.
    if t <= numDays-2:
        value_realizable = value_cl_seq[t] - np.sum(value_vec[np.logical_not(noHold)])
        new_value_vec, trans_cost = rebalance(value_vec[noHold], value_realizable, \
                                              new_portfolio[noHold], 0.0005)
        value_op_seq[t+1] = value_cl_seq[t] - trans_cost
        if value_op_seq[t+1]<0:
            sys.stderr.write('You go bankrupt at Day {}!'.format(t))
        value_vec[noHold] = new_value_vec
        portfolio_seq[:,t+1] = value_vec / value_op_seq[t+1]
        
    last_close_price[isActive] = cl[isActive]
    
    # Update information
    if t <= numDays-2:
        op = test_op[t+1,:]
        hi = test_op[t,:]
        lo = test_lo[t,:]
        vol = test_vol[t,:]

------------------------------
Results of expert pooling: 
------------------------------
Weighting strategy:  exp_window
Total dollar value of assets over time:
[ 1.          0.99950025  0.98453958  0.97936484  0.97668832  0.95646125
  0.94628035  0.94944179  0.93775277  0.94004697  0.93470971  0.95159261
  0.95962921  0.94589299  0.95598934  0.96858082  0.96955565  0.95231972
  0.94768514  0.93960302  0.93740813  0.93348141  0.9146944   0.90069733
  0.91440299  0.94803448  0.94318242  0.93464198  0.93252626  0.94465415]
Sharpe ratio:
0.975762322587


In [15]:
plt.plot(value_op_seq)
return_seq = np.log(value_op_seq[1:] / value_op_seq[:-1])
annual_return = 252 * np.mean(return_seq)
sharpe = np.sqrt(252) * np.mean(return_seq) / np.std(return_seq)
print annual_return, sharpe

NameError: name 'plt' is not defined